In [1]:
import pandas as pd
import torch
from transformers import BertForRanking
import pickle

import os
from tqdm import tqdm
import numpy as np
import seaborn as sns

In [2]:
import torch

class RankingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings1, encodings2, labels, user_id):
        self.encodings1 = encodings1
        self.encodings2 = encodings2
        self.labels = labels
        self.user_id = user_id

    def __getitem__(self, idx):
        item1 = {key + "_1": torch.tensor(val[idx]) for key, val in self.encodings1.items()}
        item2 = {key + "_2": torch.tensor(val[idx]) for key, val in self.encodings2.items()}
        item = dict(**item1, **item2)
        item['labels'] = torch.tensor(self.labels[idx])
        item['user_id'] = torch.tensor(self.user_id[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [46]:
#train_dataset = pickle.load(open("../book_data/rankingDataset/train_dataset.pkl", "rb"))
#val_dataset = pickle.load(open("../book_data/dataset/val_dataset.pkl", "rb"))
test_dataset = pickle.load(open("../book_data/rankingDataset/test_dataset.pkl", "rb"))

In [47]:

model = BertForRanking.from_pretrained("bert-base-uncased")



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForRanking: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForRanking from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForRanking from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForRanking were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.w

In [48]:
model.load_state_dict(torch.load("../results/checkpoint-1500/pytorch_model.bin"))


<All keys matched successfully>

In [49]:

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [50]:
model = model.cuda()

In [60]:
def eval_model(dataset, model, shuffle_ix=False):
    model.eval()
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=128)
    
    pred_list, real_list, uid_list = list(), list(), list()
    
    i=0
    with torch.no_grad():
        for b in tqdm(dataloader):
            if shuffle_ix:
                r=torch.randperm(len(b['user_id']))
                b['user_id'] = b['user_id'][r]
                
            b = {k:v.cuda() for k, v in b.items()}
            output = model(**b)
            
            #print(output)
            
            #out = output[1].reshape(output[1].shape[0] // 2, 2)
            out = output[1]
            #print(out)
            pred = torch.argmax(out, dim=1)
            pred_list.append(pred.cpu().numpy())
            real_list.append(b['labels'].cpu().numpy())
            uid_list.append(b['user_id'].cpu().numpy())
            
            i+=1
            if i> 10:
                break
            
            
    preds = np.concatenate(pred_list)
    reals = np.concatenate(real_list)
    uids = np.concatenate(uid_list)
    
    data = [[p, r, u] for p, r, u in zip(preds, reals, uids)]

    return pd.DataFrame(data, columns = ["pred", "read", "uid"])
    

In [56]:
df_test_rand = eval_model(test_dataset, model)

  1%|█                                                                                                                                                                      | 1/167 [00:01<02:57,  1.07s/it]

tensor(0.2629, device='cuda:0')
tensor(0.2629, device='cuda:0')
tensor([[ 0.2629, -0.2123],
        [ 0.2629,  0.2138],
        [ 0.2629, -0.1291]], device='cuda:0')


  1%|██                                                                                                                                                                     | 2/167 [00:02<02:45,  1.00s/it]

tensor(-0.4033, device='cuda:0')
tensor(-0.4033, device='cuda:0')
tensor([[-0.4033, -0.4506],
        [-0.4033,  0.4631],
        [-0.4033, -0.0791]], device='cuda:0')


  2%|███                                                                                                                                                                    | 3/167 [00:03<02:44,  1.00s/it]

...

tensor(-0.3459, device='cuda:0')
tensor(-0.3459, device='cuda:0')
tensor([[-0.3459, -0.2319],
        [-0.3459, -0.1248],
        [-0.3459, -0.3254]], device='cuda:0')


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 165/167 [02:35<00:02,  1.01s/it]

tensor(-0.1453, device='cuda:0')
tensor(-0.1484, device='cuda:0')
tensor([[-0.1453,  0.0352],
        [-0.1484,  0.2388],
        [-0.1484, -0.1272]], device='cuda:0')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 167/167 [02:36<00:00,  1.07it/s]

tensor(-0.4462, device='cuda:0')
tensor(0.2846, device='cuda:0')
tensor([[-0.4462,  0.5072],
        [ 0.2846, -0.1538],
        [ 0.2846, -0.4484]], device='cuda:0')
tensor(-0.1242, device='cuda:0')
tensor(-0.1242, device='cuda:0')
tensor([[-0.1242, -0.3340],
        [-0.1242,  0.2511],
        [-0.1242, -0.2509]], device='cuda:0')


In [57]:
df_test_rand

,pred,read,uid
0,0,1,49154
1,0,1,49154
2,0,1,49154
3,0,1,49154
4,0,1,49154
...,...,...,...
21257,0,0,172030
21258,0,0,172030
21259,1,0,172030
21260,0,0,172030


In [59]:
get_acc(df_test_rand)

0.45466089737559967

In [61]:
df_test = eval_model(train_dataset, model, True)

  0%|▏                                                                                                                                                                      | 1/774 [00:00<12:34,  1.03it/s]

tensor(-0.3393, device='cuda:0')
tensor(-0.2880, device='cuda:0')
tensor([[-0.3393,  0.2629],
        [-0.2880,  0.4195],
        [-0.2880,  0.4509]], device='cuda:0')


  0%|▍                                                                                                                                                                      | 2/774 [00:01<12:53,  1.00s/it]

tensor(0.1354, device='cuda:0')
tensor(0.1426, device='cuda:0')
tensor([[ 0.1354,  0.0239],
        [ 0.1426,  0.1169],
        [ 0.1426, -0.0152]], device='cuda:0')


  0%|▋                                                                                                                                                                      | 3/774 [00:03<13:05,  1.02s/it]

...

tensor(0.3275, device='cuda:0')
tensor(0.3275, device='cuda:0')
tensor([[ 0.3275,  0.3824],
        [ 0.3275, -0.3459],
        [ 0.3275,  0.0048]], device='cuda:0')


  1%|██▏                                                                                                                                                                   | 10/774 [00:09<12:32,  1.01it/s]

tensor(0.2613, device='cuda:0')
tensor(0.4599, device='cuda:0')
tensor([[ 0.2613, -0.2175],
        [ 0.4599, -0.2717],
        [ 0.4599,  0.0909]], device='cuda:0')


  1%|██▏                                                                                                                                                                   | 10/774 [00:10<13:55,  1.09s/it]

tensor(0.3066, device='cuda:0')
tensor(0.3542, device='cuda:0')
tensor([[ 0.3066, -0.1900],
        [ 0.3542, -0.2466],
        [ 0.3542, -0.2247]], device='cuda:0')


In [62]:
df_test

,pred,read,uid
0,1,0,49154
1,1,0,32773
2,1,0,32773
3,1,0,32773
4,1,0,32773
...,...,...,...
1403,1,0,245864
1404,1,0,139387
1405,1,0,278633
1406,1,0,245864


In [63]:
get_acc(df_test)

0.06676136363636363

In [ ]:
get_acc(df_test)

In [ ]:
df_train = eval_model(train_dataset, model)

In [13]:
def get_acc(df: pd.DataFrame)->float:
    return (df.pred == df.read).sum() / df.shape[0]

In [ ]:
train_acc = (df_train.pred == df_train.read).sum() / df_train.shape[0]
test_acc = (df_test.pred == df_test.read).sum() / df_test.shape[0]

print(train_acc, test_acc)

In [ ]:
train_count = df_train.uid.value_counts()
test_count = df_test.uid.value_counts()

In [ ]:
train_count = train_count[train_count.index.isin(test_count.index)]

In [ ]:
train_count = train_count.sort_index()
test_count = test_count.sort_index()

In [ ]:
all(train_count.index == test_count.index)

In [ ]:
all_counts = pd.concat([train_count, test_count],1)

In [ ]:
all_counts = all_counts.set_axis(["trainCount", "testCount"], axis=1)

In [ ]:
all_counts

In [ ]:
items = list()
for i, row in all_counts.iterrows():
    acc = get_acc(df_test[df_test.uid == i])
    item = list(row) + [acc, i]
    items.append(item)


In [ ]:
result_df = pd.DataFrame(items, columns=["trainCount", "testCount", "acc", "uid"])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('ggplot')
sns.set_style("whitegrid", {'axes.grid' : False})


In [ ]:
plt.figure(figsize=(10, 6))

ax = sns.histplot(result_df.acc)

ax.invert_xaxis()

#plt.legend(title='', loc='upper right', labels=['Prosit Transformer', 'Prosit RNN'], prop={"size":14})

A = ax.get_legend()
#A.set_title('')

#plt.setp(A.get_texts(), fontsize='14') # for legend text
#plt.setp(A.get_title(), fontsize='14') # for legend title

plt.xlabel("Accuracy", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Number of Users", fontsize=14)
plt.tight_layout()

#seaborn.histplot(data=filterDf(datafile), x="Angular Similarity", hue="Data Set", alpha=0.2)
#plt.savefig('./plots/spectralAngleDist.png')

In [ ]:
plt.figure(figsize=(10, 6))

ax = sns.histplot(result_df[result_df.trainCount>20].acc)

ax.invert_xaxis()

#plt.legend(title='', loc='upper right', labels=['Prosit Transformer', 'Prosit RNN'], prop={"size":14})

A = ax.get_legend()
#A.set_title('')

#plt.setp(A.get_texts(), fontsize='14') # for legend text
#plt.setp(A.get_title(), fontsize='14') # for legend title

plt.xlabel("Accuracy", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.tight_layout()

#seaborn.histplot(data=filterDf(datafile), x="Angular Similarity", hue="Data Set", alpha=0.2)
#plt.savefig('./plots/spectralAngleDist.png')

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.histplot(result_df[(result_df.trainCount>10) & (result_df.testCount>5)].acc)

ax.invert_xaxis()

#plt.legend(title='', loc='upper right', labels=['Prosit Transformer', 'Prosit RNN'], prop={"size":14})

A = ax.get_legend()
#A.set_title('')

#plt.setp(A.get_texts(), fontsize='14') # for legend text
#plt.setp(A.get_title(), fontsize='14') # for legend title

plt.xlabel("Accuracy", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("Number of Users", fontsize=14)
plt.tight_layout()

#seaborn.histplot(data=filterDf(datafile), x="Angular Similarity", hue="Data Set", alpha=0.2)
#plt.savefig('./plots/spectralAngleDist.png')

In [ ]:
ax = sns.distplot(result_df.acc, hist=False)

In [ ]:
result_df

In [ ]:
row

In [ ]:
ixs_1 = train_count[train_count == 1].index

In [ ]:
get_acc(df_test[df_test.uid.isin(ixs_1)])

In [ ]:
def get_ixs(counts, l, b=None):
    if b is not None:
        return counts[(l <= counts) & (counts <= b)].index
    else:
        return counts[counts == l].index
    
def filerDf(df, train_ix, test_ix):
    df = df[df.uid.isin(train_ix)]
    df = df[df.uid.isin(test_ix)]
    return df
    
def getUserAcc(df):
    acc_list = list()
    for uid in df_10.uid:
        acc = get_acc(df[df.uid == uid])
        acc_list.append(acc)
    return acc_list
  

In [ ]:
train_ixs_10 = get_ixs(train_count, 5, 10) 
test_ixs_10 = get_ixs(test_count, 5, 10) 
df_10 = filerDf(df, train_ixs_10, test_ixs_10)

In [ ]:
acc_list = getUserAcc(df_10)
sns.boxplot(acc_list)

In [ ]:
train_ixs_10 = get_ixs(train_count, 10, 100) 
test_ixs_10 = get_ixs(test_count, 10, 100) 
df_10 = filerDf(df, train_ixs_10, test_ixs_10)
acc_list = getUserAcc(df_10)
sns.boxplot(acc_list)

In [ ]:
train_ixs_10 = get_ixs(train_count, 100, 1000) 
test_ixs_10 = get_ixs(test_count, 1, 1000) 
df_10 = filerDf(df, train_ixs_10, test_ixs_10)
acc_list = getUserAcc(df_10)
sns.boxplot(acc_list)

In [ ]:
get_acc(df_10[df_10.uid == uid])

In [ ]:
get_acc(df_test[df_test.uid.isin(ixs_10)])

In [ ]:
ixs_100 = train_count[train_count >= 2000].index

In [ ]:
get_acc(df_test[df_test.uid.isin(ixs_100)])

In [ ]:
df_test.uid.isin(ixs_100).sum()

In [ ]:
ixs_100

In [ ]:
val_count = df_train.uid.value_counts()

In [ ]:
preds = np.concatenate(pred_list)
reals = np.concatenate(real_list)
uids = np.concatenate(uid_list)

In [ ]:
data = [[p, r, u] for p, r, u in zip(preds, reals, uids)]

df = pd.DataFrame(data, columns = ["pred", "read", "uid"])

In [ ]:
val_count = df.uid.value_counts()

In [ ]:
counts = (val_count == 1)


In [ ]:
ix = counts[counts.values].index

In [ ]:
df_1_review = df[df.uid.isin(ix)]

In [ ]:
df_1_review

In [ ]:
(df_1_review.pred == df_1_review.read).sum() / df_1_review.shape[0]

In [ ]:
preds = torch.cat(pred_list)
reals = torch.cat(real_list)

In [ ]:
(preds == reals).sum() / reals.shape[0]